In [1]:
from compute import Var, Par
from datastructures.api import Model, adda, addf, addsolver, setsolvefor, addobj, addeq, addineq, OPT
from datastructures.execution import edges_from_components
from datastructures.graphutils import namefromsympy, dfs_tree, SOLVER, VAR, COMP, nested_sources, sources, edges_to_Ein_Eout, flat_graph_formulation, root_solver
from datastructures.nestedgraph import root_sources
from datastructures.execution import generate_components_and_residuals
from datastructures.workflow import get_f, order_from_tree, default_solver_options, mdao_workflow, implicit_comp_name
from datastructures.workflow_mdao import mdao_workflow_with_args
from datastructures.operators import sort_scc, reorder_merge_solve
from datastructures.assembly import build_archi
from datastructures.api import evalexpr
from datastructures.postprocess import print_inputs, print_outputs, update_varval
from datastructures.rendering import render_incidence
import openmdao.api as om
import numpy as np
from sympy import cos
import matplotlib.pyplot as plt

In [2]:
from compute import ureg
ureg.define('USD = [currency] ')

# Variables and parameters 
They are shared accross models

In [3]:
P_t = Var('P_c', unit='W')
P_t.forceunit=True
Pbus = Var('P_b', unit='W')
Pbus.forceunit=True
S = Var('h', 400, unit='km')
D = Var('D', 0.1, unit='m')
A = Var('A', 0.5, unit='m^2')
eta_t = Par(r'\eta_t', 0.16)
eta_a = Par(r'\eta', 0.5)
alpha = Par(r'\alpha', 1e20, 'bits/W')
th = Par(r'\theta', np.pi/4)
W0 = Par('W_0', 1367, 'W/m^2')
Re = Par('Re', 6378, 'km')
μ = Par('mu', 3.986005e14, 'm^3/s^2')
c1 = Par('c_1', 2500, 'USD/m^2')
c2 = Par('c_2', 12000, 'USD/m^2')
c3 = Par('c_3', 100, 'USD/W')
c4 = Par('c_4', 12000, 'USD')

# Formulation 1

In [4]:
model = Model(solver=OPT)
m = model.root
s2 = addsolver(m)#, solvefor=['Pbus', 'P_t'])
adda(s2, P_t, A*eta_a*W0*cos(th)-Pbus)
adda(s2, Pbus, 10*abs(P_t)**0.5)
R = adda(m, 'R', alpha*P_t*D**2*eta_t/(16*S**2), unit='MB/s', forceunit=True)
C = adda(m, 'C', c1*D**2+c2*A+c3*Pbus+c4, unit='kUSD')
addeq(m, -R+1) # == 0
addobj(m, C)

In [5]:
from compute import ureg

In [20]:
evalexpr(alpha*eta_t/(16)).varval, evalexpr(eta_a*W0*cos(th)).varval

(array(1.e+18), array(483.30748494))

In [7]:
setsolvefor(m, [D,A], {D:[0.5,10], A:[0.04,1]})

In [8]:
edges, tree = model.generate_formulation()

In [9]:
Ein, Eout, Rin = edges

In [10]:
# Ein_noparam = {comp:tuple(var for var in compvars if not model.idmapping[var].always_input) for comp,compvars in Ein.items()}
# edges_noparam = Ein_noparam, Eout, Rin
#_, ax = render_incidence(edges_noparam, tree, namingfunc)

In [11]:
ordered_tree = tree
ordered_edges = edges
comp_options = model.comp_options
var_options = model.var_options
solvers_options = model.solvers_options
solvers_options[1].update({'optimizer': 'SLSQP'})
solvers_options[2] = {'solver': 'N', 'maxiter':20}
nodetyperepr = model.nametyperepr
namingfunc = namefromsympy(nodetyperepr)
sequence = order_from_tree(ordered_tree[0], ordered_tree[1], ordered_edges[1])
solvers_options = default_solver_options(ordered_tree, solvers_options)
wf = mdao_workflow(sequence, solvers_options, comp_options, var_options)
components = model.components
lookup_f = get_f(components, ordered_edges)
wfmdao = mdao_workflow_with_args(wf, lookup_f, namingfunc)

In [35]:
components

[(('\\theta', 'P_b', '\\eta', 'A', 'W_0'), 0, ('P_c',), 'W_0*\\eta*A*cos(\\theta) - P_b'),
 (('P_c',), 1, ('P_b',), '10*Abs(P_c)**0.5'),
 (('\\eta_t', '\\alpha', 'D', 'P_c', 'h'), 2, ('R',), '\\alpha*\\eta_t*D**2*P_c/(16*h**2)'),
 (('c_1', 'D', 'c_3', 'P_b', 'c_4', 'A', 'c_2'), 3, ('C',), 'c_1*D**2 + c_2*A + c_3*P_b + c_4'),
 (('R',), 4, (None,), '1 - R'),
 (('C',), 5, (None,), 'C')]

In [38]:
components[0].graddict({'\\theta':th.varval, 'P_b':10, '\\eta':eta_a.varval, 'A':10, 'W_0':W0.varval})

{('P_c', '\\theta'): DeviceArray(-4833.07484941, dtype=float64),
 ('P_c', 'P_b'): DeviceArray(-1., dtype=float64),
 ('P_c', '\\eta'): DeviceArray(9666.14969882, dtype=float64),
 ('P_c', 'A'): DeviceArray(483.30748494, dtype=float64),
 ('P_c', 'W_0'): DeviceArray(3.53553391, dtype=float64)}

In [39]:
components[2].graddict({'\\eta_t':eta_t.varval, '\\alpha':alpha.varval, 'D':10, 'P_c':10, 'h':S.varval})

{('R', '\\eta_t'): DeviceArray(4882.8125, dtype=float64),
 ('R', '\\alpha'): DeviceArray(7.8125e-18, dtype=float64),
 ('R', 'D'): DeviceArray(156.25, dtype=float64),
 ('R', 'P_c'): DeviceArray(78.125, dtype=float64),
 ('R', 'h'): DeviceArray(-3.90625, dtype=float64)}

In [41]:
components[3].graddict({'c_1':c1.varval, 'D':10, 'c_3':c3.varval, 'P_b':10, 'c_4':c4.varval, 'A':10, 'c_2':c2.varval})

{('C', 'c_1'): DeviceArray(0.1, dtype=float64),
 ('C', 'D'): DeviceArray(50., dtype=float64),
 ('C', 'c_3'): DeviceArray(0.01, dtype=float64),
 ('C', 'P_b'): DeviceArray(0.1, dtype=float64),
 ('C', 'c_4'): DeviceArray(0.001, dtype=float64),
 ('C', 'A'): DeviceArray(12., dtype=float64),
 ('C', 'c_2'): DeviceArray(0.01, dtype=float64)}

In [12]:
#evalexpr(A*eta_a*W0*cos(th))

In [46]:
prob, mdao_in, groups = build_archi(ordered_edges, ordered_tree, wfmdao, namingfunc, model.idmapping)

In [59]:
prob.set_val('D', 0.67)
prob.set_val('A', 0.1)

In [60]:
prob.run_model();


==
s2
==
NL: Newton Converged in 4 iterations


In [61]:
print_outputs(model, prob, namingfunc, rounding=3)

$$P_c$$,12.697,W
$$P_b$$,35.633,W
$$R$$,4.453,MB/s
$$C$$,17.886,kUSD


In [62]:
#print_inputs(model, prob, namingfunc, mdao_in)

In [63]:
#print_outputs(model, prob, namingfunc, rounding=1)

In [64]:
prob.run_driver()


==
s2
==
NL: Newton Converged in 0 iterations

==
s2
==
NL: Newton Converged in 0 iterations

==
s2
==
NL: Newton Converged in 5 iterations

==
s2
==
NL: Newton Converged in 4 iterations

==
s2
==
NL: Newton Converged in 4 iterations

==
s2
==
NL: Newton Converged in 2 iterations

==
s2
==
NL: Newton Converged in 3 iterations

==
s2
==
NL: Newton Converged in 3 iterations

==
s2
==
NL: Newton Converged in 3 iterations

==
s2
==
NL: Newton Converged in 2 iterations
Optimization terminated successfully    (Exit mode 0)
            Current function value: 15.300923260384632
            Iterations: 8
            Function evaluations: 9
            Gradient evaluations: 7
Optimization Complete
-----------------------------------


False

In [21]:
print_inputs(model, prob, namingfunc, mdao_in)

$$D$$,0.67,m
$$h$$,400,km
$$A$$,0.041,m²


In [25]:
prob.get_val('D'), prob.get_val('A')

(array([0.67048414]), array([0.04080474]))

In [22]:
print_outputs(model, prob, namingfunc, rounding=1)

$$P_c$$,2.8,W
$$P_b$$,16.9,W
$$R$$,1,MB/s
$$C$$,15.3,kUSD


In [17]:
import numpy as np

In [28]:
Pc, Pb = 18.32, 42.80186912

In [38]:
np.array([Pc,Pb])-np.linalg.solve(np.array([[1,1],[-10/(2*np.sqrt(Pc)), 1]]), [Pc+Pb-19.332,Pb-10*np.sqrt(Pc)])

C:\Users\johan\AppData\Local\Temp/ipykernel_16872/3458300415.py:1: RuntimeWarning: invalid value encountered in sqrt
  np.array([Pc,Pb])-np.linalg.solve(np.array([[1,1],[-10/(2*np.sqrt(Pc)), 1]]), [Pc+Pb-19.332,Pb-10*np.sqrt(Pc)])


array([nan, nan])

In [30]:
Pc, Pb = -0.95422934, 20.28622934

In [49]:
prob.run_model()


==
s1
==
P_c Pbus {'Pbus': array([42.80186912]), 'P_0': array([19.32])} -1.0
P_c P_0 {'Pbus': array([42.80186912]), 'P_0': array([19.32])} 1.0
Pbus P_c {'P_c': array([18.32])} 1.1681732837899945
P_c Pbus {'Pbus': array([9.79675432]), 'P_0': array([19.32])} -1.0
P_c P_0 {'Pbus': array([9.79675432]), 'P_0': array([19.32])} 1.0
Pbus P_c {'P_c': array([-0.95976395])} -5.103731131122634
P_c Pbus {'Pbus': array([18.74638868]), 'P_0': array([19.32])} -1.0
P_c P_0 {'Pbus': array([18.74638868]), 'P_0': array([19.32])} 1.0
Pbus P_c {'P_c': array([-3.51427089])} -2.6671803751021677
P_c Pbus {'Pbus': array([24.42589701]), 'P_0': array([19.32])} -1.0
P_c P_0 {'Pbus': array([24.42589701]), 'P_0': array([19.32])} 1.0
Pbus P_c {'P_c': array([-5.96624445])} -2.0470077300787213
P_c Pbus {'Pbus': array([26.05372273]), 'P_0': array([19.32])} -1.0
P_c P_0 {'Pbus': array([26.05372273]), 'P_0': array([19.32])} 1.0
Pbus P_c {'P_c': array([-6.78796468])} -1.9191115417404778
P_c Pbus {'Pbus': array([26.1667352

In [18]:
recorder = om.SqliteRecorder('cases2.sqlite')

In [19]:
prob.model.s2.add_recorder(recorder)
for key,val in ordered_tree[0].items():
    s = groups[namingfunc(val, SOLVER)]
    f = getattr(s, namingfunc(key, COMP))
    f.recording_options['options_excludes'] = ['component']

In [21]:
prob.cleanup();

In [22]:
cr = om.CaseReader("cases2.sqlite")
solver_cases = cr.list_cases(out_stream=None);

In [25]:
cr.get_case(solver_cases[0]).get_val('P_b')

array([nan])

In [166]:
#print_outputs(model, prob, namingfunc)

In [140]:
prob.run_driver();


==
s2
==
NL: Newton Converged in 3 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 1 iterations

==
s2
==
NL: Newton Converged in 4 iterations

==
s2
==
NL: Newton Converged in 4 iterations

==
s2
==
NL: Newton Converged in 3 iterations

==
s2
==
NL: Newton Converged in 3 iterations

==
s2
==
NL: Newton Converged in 3 iterations

==
s2
==
NL: Newton Converged in 2 iterations
Optimization terminated successfully    (Exit mode 0)
            Current function value: [15.30092333]
            Iterations: 12
            Function evaluations: 13
            Gradient evaluations: 9
Optimization Complete
-----------------------------------


In [16]:
mdao_in_dv = mdao_in.union({'D','A'})

In [54]:
print_inputs(model, prob, mdao_in_dv)

NameError: name 'mdao_in_dv' is not defined

In [55]:
print_outputs(model, prob, namingfunc)

,0,1,2
0,P_c,2.75,W
1,P_b,16.583,W
2,R,0.908,MB
3,C,15.195,kUSD


In [15]:
print(print_inputs(model, mdao_in).style.to_latex())

\begin{tabular}{llr}
 & 0 & 1 \\
0 & A & 0.500000 \\
1 & D & 0.100000 \\
2 & S & 400.000000 \\
\end{tabular}



In [16]:
print(print_outputs(model, prob, namingfunc).style.to_latex())

\begin{tabular}{llll}
 & 0 & 1 & 2 \\
0 & C & 26.063 & kUSD \\
1 & P_b & 80.381 & W \\
2 & P_t & 64.611 & W \\
3 & R & 0.946 & MB \\
\end{tabular}



In [10]:
update_varval(model, prob, namingfunc)

In [11]:
evalexpr(10*P_t**0.5)

dummy

# Restructured

In [20]:
modelrstr = Model(solver=OPT)
mrstr = modelrstr.root
R = Var('R', unit='MB/s')
adda(mrstr, P_t, R/(alpha*D**2*eta_t)*(16*S**2))
adda(mrstr, Pbus, 10*P_t**0.5)
adda(mrstr, A, (Pbus+P_t)/(eta_a*W0*cos(th)))
C = adda(mrstr, 'C', c1*D**2+c2*A+c3*Pbus+c4, unit='kUSD')
addobj(mrstr, C)

In [21]:
setsolvefor(mrstr, [D], {D:[0,10]})

In [22]:
edges, tree = modelrstr.generate_formulation()

In [23]:
ordered_tree = tree
ordered_edges = edges
comp_options = modelrstr.comp_options
var_options = modelrstr.var_options
solvers_options = modelrstr.solvers_options
#solvers_options[1] = {'solver': 'N', 'maxiter':20}
nodetyperepr = modelrstr.nametyperepr
namingfunc = namefromsympy(nodetyperepr)
sequence = order_from_tree(ordered_tree[0], ordered_tree[1], ordered_edges[1])
solvers_options = default_solver_options(ordered_tree, solvers_options)
wf = mdao_workflow(sequence, solvers_options, comp_options, var_options)
components = modelrstr.components
lookup_f = get_f(components, ordered_edges)
wfmdao = mdao_workflow_with_args(wf, lookup_f, namingfunc)

In [36]:
prob, mdao_in, groups = build_archi(ordered_edges, ordered_tree, wfmdao, namingfunc, modelrstr.idmapping)

In [37]:
R.varval = 1
prob.set_val('R', 1.003)
#D.varval = 0.001
#prob.set_val('D', 0.43057)

In [38]:
prob.run_model()

In [35]:
prob.run_driver();

Optimization terminated successfully    (Exit mode 0)
            Current function value: [15.30429158]
            Iterations: 5
            Function evaluations: 9
            Gradient evaluations: 5
Optimization Complete
-----------------------------------


In [25]:
mdao_in_dv = mdao_in.union({'D'})

In [39]:
print_inputs(modelrstr, prob, namingfunc, mdao_in)

$$D$$,0.1,m
$$h$$,400,km
$$R$$,1.003,MB/s


In [40]:
print_outputs(modelrstr, prob, namingfunc, rounding=1)

$$P_c$$,128.4,W
$$P_b$$,113.3,W
$$A$$,0.5,m²
$$C$$,29.4,kUSD


In [81]:
update_varval(modelrstr, prob, namingfunc)